In [284]:
import pandas as pd
import numpy as np

## mapping 
import folium

## Plot
import matplotlib.pyplot as plt

In [285]:
con_df = pd.read_csv('./Condominium_comparable_rental_income.csv')
grn_df = pd.read_csv('./Greenthumb_Community_Gardens.csv')

In [286]:
con_df = con_df.dropna()

## Mapping 

In [287]:
grn_df['Latitude'] = grn_df['Latitude'].astype(float)
grn_df['Longitude'] = grn_df['Longitude'].astype(float)
grn_df = grn_df.dropna(axis=0, subset=['Latitude','Longitude'])

In [288]:
con_df['Latitude'] = con_df['Latitude'].astype(float)
con_df['Longitude'] = con_df['Longitude'].astype(float)
con_df = con_df.dropna(axis=0, subset=['Latitude','Longitude'])

In [289]:
map_1 = folium.Map(location=[40.773752, -73.972679], zoom_start = 12) ## ["Stamen Terrain", "Stamen Toner", "Mapbox Bright"]

housings_points = int(con_df['Longitude'].describe()['count'])
gardens_points = int(grn_df['Longitude'].describe()['count'])


for i in range(housings_points):
    # Other areas   
    latitude = con_df.iloc[i]['Latitude']
    longitude = con_df.iloc[i]['Longitude']
    
    folium.CircleMarker([latitude, longitude],
                        radius=2,
                        color='#FF0000',
                        fill_color='#FF0000', ).add_to(map_1)


## green areas  
for i in range(gardens_points):
    latitude = grn_df.iloc[i]['Latitude']
    longitude = grn_df.iloc[i]['Longitude']
    
    folium.CircleMarker([latitude, longitude],
                        radius=2,
                        color='#008000',
                        fill_color='#008000', ).add_to(map_1)
    
map_1

### Total units heat map

In [290]:
np.unique(con_df['Postcode'])

array([ 10000.,  10001.,  10002.,  10003.,  10004.,  10005.,  10006.,
        10007.,  10009.,  10010.,  10011.,  10012.,  10013.,  10014.,
        10016.,  10017.,  10018.,  10019.,  10021.,  10022.,  10023.,
        10024.,  10025.,  10026.,  10027.,  10028.,  10029.,  10030.,
        10031.,  10032.,  10033.,  10034.,  10035.,  10036.,  10037.,
        10038.,  10039.,  10040.,  10065.,  10069.,  10075.,  10128.,
        10280.])

In [291]:
postcodes = con_df.groupby('Postcode', as_index = False).agg(np.mean)

In [292]:
postcodes["Postcode"] = postcodes["Postcode"].apply(lambda x: str(int(x)))

In [293]:
postcodes["Postcode"] = postcodes["Postcode"].astype(int)

In [270]:
def show_postcode_map(col):
    
    postcode_map = folium.Map(location=[40.773752, -73.972679], tiles = "cartodbpositron", zoom_start = 12)
     
    postcode_map.choropleth(
         geo_data = 'nyczipcode.geojson',
         data = postcodes,
         columns=['Latitude', col],
         key_on='feature.properties.postalCode',
         fill_color='YlGn',
         fill_opacity=0.7,
         line_opacity=0.2,
         legend_name='Unemployment Rate (%)',
         reset=True
     )
    
    for i in range(gardens_points):
        latitude = grn_df.iloc[i]['Latitude']
        longitude = grn_df.iloc[i]['Longitude']

        folium.CircleMarker([latitude, longitude],
                            radius=2,
                            color='#008000',
                            fill_color='#008000', ).add_to(postcode_map)
    return postcode_map

In [271]:
show_postcode_map('MANHATTAN CONDOMINIUM PROPERTY Est. Gross Income')

In [205]:
import branca

In [295]:
print(postcodes.dtypes)

Postcode                                                  int64
Borough                                                 float64
Latitude                                                float64
Longitude                                               float64
Community Board                                         float64
Council District                                        float64
Census Tract                                            float64
BIN                                                     float64
BBL                                                     float64
MANHATTAN CONDOMINIUM PROPERTY Total Units              float64
MANHATTAN CONDOMINIUM PROPERTY Year Built               float64
MANHATTAN CONDOMINIUM PROPERTY Gross SqFt               float64
MANHATTAN CONDOMINIUM PROPERTY Est. Gross Income        float64
MANHATTAN CONDOMINIUM PROPERTY Gross Income per SqFt    float64
MANHATTAN CONDOMINIUM PROPERTY Full Market Value        float64
MANHATTAN CONDOMINIUM PROPERTY Market Va

In [296]:
minc = postcodes['MANHATTAN CONDOMINIUM PROPERTY Est. Gross Income'].describe()['min']
maxc = postcodes['MANHATTAN CONDOMINIUM PROPERTY Est. Gross Income'].describe()['max']

In [297]:
colorscale = branca.colormap.linear.YlGnBu.scale(minc, maxc)

In [299]:
postcodes[postcodes['Postcode'] == 10075]

,Postcode,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,MANHATTAN CONDOMINIUM PROPERTY Total Units,...,COMPARABLE RENTAL 1 Market Value per SqFt,COMPARABLE RENTAL 1 Dist. from Coop in miles,COMPARABLE RENTAL 2 Total Units,COMPARABLE RENTAL 2 Year Built,COMPARABLE RENTAL 2 Gross SqFt,COMPARABLE RENTAL 2 Est. Gross Income,COMPARABLE RENTAL 2 Gross Income per SqFt,COMPARABLE RENTAL 2 Full Market Value,COMPARABLE RENTAL 2 Market Value per SqFt,COMPARABLE RENTAL 2 Dist. from Coop in miles
40,10075,1.0,40.774001,-73.957052,8.0,4.434783,137.130435,1.058516e+06,1.014721e+09,106.521739,...,187.695652,0.508696,105.0,1949.347826,101766.173913,4.033450e+06,40.208696,1.939913e+07,192.956522,0.56087


In [272]:
def my_color_function(feature):
    """Maps low values to green and hugh values to red."""
    postal_code = feature['properties']['postalCode']
    aa = postcodes[postcodes['Postcode'] == int(postal_code)]['MANHATTAN CONDOMINIUM PROPERTY Est. Gross Income']
    
    if len(aa) == 0:
        return '#white'
    else:
        cc = colorscale(float(aa))
        print(cc)
        return cc

In [273]:

postcode_map = folium.Map(location=[40.773752, -73.972679], tiles = "cartodbpositron", zoom_start = 12)

colorscale = branca.colormap.linear.YlOrRd.scale(minc, maxc)
feature = postcodes['MANHATTAN CONDOMINIUM PROPERTY Est. Gross Income'].astype(int)

folium.GeoJson(
    'nyczipcode.geojson',
    style_function = lambda feature: {
        'fillColor': my_color_function(feature),
        'color' : 'black',
        'weight' : 2,
        'dashArray' : '5, 5'
        }
    ).add_to(postcode_map)

postcode_map

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
